# <font style="color:blue">Non-Maximum Suppression (NMS)</font>

In many cases, detection algorithms do not output a detection for every GT object. Instead, they create a cloud of detections that cover the whole object or some parts of it. For example, in the Sliding Window approach, the classifier can recognise the object even if the window only partially covers the object.


Just like classification, in detection too, the boxes have corresponding confidence scores. Ideally, the central detection has the highest score, and  the further the boxes are from central detection, the lower is their score.

To compress these boxes into a single good detection, we need to apply `Non-Maximum Suppression` algorithm, or `NMS`. Imagine we have a confident detection that best covers the object, and some less-confident ones that highly overlap this first one. In this case, NMS will keep the most-confident detection, and suppress the less-confident ones. It's important that the less-confident detections have a high overlap with the confident one., Otherwise, NMS won't suppress them, and they will become false positives.

NMS, or some modification of it is used as a post-processing step in most modern Deep Learning approaches.

## <font style="color:green">1. NMS Pipeline</font>

The algorithm inputs a list of the detected ($dbox\_list$) boxes with scores ($dbox\_scores\_list$).
You also need to select a suppression threshold ($threshold$).


We have already covered **Intersection over Union metric**.
For two boxes, it measures the ratio of the area of their intersection to the area of their union.
Here we will use it to select the boxes to suppress.

<br>The algorithm::</br>
1. Sort the detected boxes by confidence score.
2. Select the detected box with the current maximum score ($max\_score\_dbox$).
3. Remove this box from the $dbox\_list$.
4. Suppress other detected boxes that highly overlap this box. By high
overlap, we mean that the Intersection over Union value is higher than the
suppression threshold value.
5. Repeat steps `2-5` for other boxes left in $dbox\_list$.


## <font style="color:green">2. Soft-NMS</font>

NMS is a pretty straightforward algorithm. There are but several add-on tricks that vastly improve its quality - and one of them is Soft-NMS.

The original Soft-NMS article [Improving Object Detection With One Line of Code](https://arxiv.org/pdf/1704.04503.pdf) reveals that NMS has some drawbacks. One of them is that the two GT boxes overlap, and NMS can remove the less confident one. Also, even though the algorithm correctly detected both objects, we'll ignore one of them, which will become a false negative. The picture below illustrates two bounding boxes of horses with high overlap and different scores: red is `0.95` and green is `0.80`.

---

<img src='https://www.learnopencv.com/wp-content/uploads/2020/03/c3-w8-bboxes_overlap.png' align='middle' width=700>

---

NMS will remove the green box, and set its confidence score to 0. `Soft-NMS` algorithm slightly modifies the NMS steps introducing

1. new re-scoring method
2. updated suppression policy 

Let's check out  the steps of both the algorithm versions:


---

<img src='https://www.learnopencv.com/wp-content/uploads/2020/03/c3-w8-nms_algorithms.jpg' align='middle' width=400>

---

In the red box, you can see the original NMS steps, and in the green box - Soft NMS. Now let's pay attention to the score recalculation. In NMS, it can be written as:

$$
s_i = \left\{
\begin{array}{ll}
0{, } & iou(\mathcal{M},b_i) \geqslant N_t\textrm{,}\\
s_i{, } & iou(\mathcal{M},b_i) < N_t\textrm{.}\\
\end{array} \right.
$$

It turns out that the box exclusion rule is quite hard: If IoU is greater than the predefined threshold value, the box will be removed and its confidence score will be set to zero. However, it can be softened, by decreasing only the score of the box with high $\mathcal{M}$ (detected box with a maximum score) overlap:


$$
s_i = \left\{
\begin{array}{ll}
s_i(1 - iou(\mathcal{M},b_i)){, } & iou(\mathcal{M},b_i) \geqslant N_t\textrm{,}\\
s_i{, } & iou(\mathcal{M},b_i) < N_t\textrm{.}\\
\end{array} \right.
$$

The described rescoring function is a **linear** $\mathcal{M}$ overlap function. The bounding boxes distant from $\mathcal{M}$ won't be affected, whereas the ones closest will be highly affected. The decrease in the scores should be proportional to the overlap level. When the box overlap with $\mathcal{M}$ is close to one, it should be "punished". There is a **Gaussian** "punishment" function, which involves these terms:


$$
s_i = s_ie^{-\frac{iou(\mathcal{M},b_i)^{2}}{\sigma}}, \forall b_i \notin \mathcal{D}
$$

where $\mathcal{D}$ - final detected boxes.

### <font style="color:green">Soft-NMS Implementation</font>

**Let's implement the Soft-NMS algorithm in line with the steps described above and in the  [Soft-NMS article](https://arxiv.org/pdf/1704.04503.pdf):**


- Rescoring using linear or gaussian method


- Speed up the computations using an appropriate PyTorch CUDA API

In [1]:
import torch

In [2]:
def soft_nms_rescoring(dbox_data, sigma=0.5, iou_threshold=0.3, score_threshold=0.001, rescoring=1):
    """
    Soft-NMS Pytorch implementation

    Parameters:
        dbox_data: coordinates of the detected boxes and their scores [x1, y1, x2, y2, confidence_score_0]
        sigma: parameter of the Gaussian function
        iou_threshold: Intersection over Union threshold (for the linear method)
        score_threshold: confidence score threshold
        rescoring: an integer value 0 or 1, where 1 corresponds to Gaussian rescoring method and 0 
                   to the linear method

    Return value:
        an array of bounding boxes with corresponding recalculated scores (in accordance with the applied method)
    """

    device = dbox_data.device

    # get bounding box coordinates
    x1_initial = dbox_data[:, 0]
    y1_initial = dbox_data[:, 1]
    x2_initial = dbox_data[:, 2]
    y2_initial = dbox_data[:, 3]

    # calculating an area of detection boxes
    areas = (x2_initial - x1_initial + 1) * (y2_initial - y1_initial + 1)

    # concatenate area of boxes with dbox_data tensor
    dbox_data = torch.cat((dbox_data, areas[:, None]), dim=1)

    final_dbox = []

    while dbox_data.shape[0] > 0:
        # position of detection box with maximum confidence score
        max_index = torch.argmax(dbox_data[:, 4], axis=0)

        # interchange current bounding box with a max score box
        dbox_data[[0, max_index], :] = dbox_data[[max_index, 0], :]

        # add max score box to the result
        final_dbox.append(dbox_data[0, :-1])

        # identifying overlap box coordinates
        xx1 = torch.max(dbox_data[0, 0], dbox_data[1:, 0])
        yy1 = torch.max(dbox_data[0, 1], dbox_data[1:, 1])
        xx2 = torch.min(dbox_data[0, 2], dbox_data[1:, 2])
        yy2 = torch.min(dbox_data[0, 3], dbox_data[1:, 3])

        # get size of overlap sides
        x_diff = xx2 - xx1 + 1
        y_diff = yy2 - yy1 + 1

        width = torch.max(x_diff, torch.tensor(0.0, device=device))
        height = torch.max(y_diff, torch.tensor(0.0, device=device))

        # IoU calculation
        intersection_area = width * height
        iou = intersection_area / (dbox_data[0, 5] + dbox_data[1:, 5] - intersection_area)

        # score recalculation with different methods
        if rescoring == 0:
            score = torch.ones(iou.shape, device=device)
            score[iou > iou_threshold] *= (1 - iou[iou > iou_threshold])

        elif rescoring == 1:
            score = torch.exp(-(iou * iou) / sigma)
        dbox_data[1:, 4] *= score

        final_box_pos = torch.where(dbox_data[1:, 4] >= score_threshold)[0]
        dbox_data = dbox_data[final_box_pos + 1, :]

    return torch.stack(final_dbox)

### <font style="color:green">System Settings for CUDA</font>

In [3]:
def setup_system() -> None:
    """
        System settings if CUDA was enabled
    """
    torch.backends.cudnn_benchmark_enabled = True
    torch.backends.cudnn.deterministic = True

### <font style="color:green">Check Soft-NMS Implementation</font>

This function will prepare inputs and call for our `Soft-NMS` implementation.

In [4]:
def check_soft_nms():
    # CUDA flag
    device_to_use = "cuda"

    # bounding boxes and the appropriate confidence scores
    # [x1, y1, x2, y2, confidence_score_0]
    detection_boxes_data = torch.tensor([[2, 2, 5, 6, 0.8], [3, 1, 5, 5, 0.1], [4, 4, 6, 7, 0.9]],
                                        dtype=torch.float)

    detection_boxes_data_2 = torch.tensor([[100., 100., 500., 400., 0.85], [450., 350., 700., 600., 0.45],
                                           [600., 100., 800., 300., 0.2]],
                                          dtype=torch.float)

    # CUDA settings
    if device_to_use == "cuda" and torch.cuda.is_available():
        setup_system()
        detection_boxes_data = detection_boxes_data.to(device_to_use)
        detection_boxes_data_2 = detection_boxes_data_2.to(device_to_use)

    # Linear
    print('Linear')
    print(soft_nms_rescoring(dbox_data=detection_boxes_data, rescoring=0))
    print(soft_nms_rescoring(dbox_data=detection_boxes_data_2, rescoring=0))

    # Gaussian
    print('Gaussian')
    print(soft_nms_rescoring(dbox_data=detection_boxes_data))
    print(soft_nms_rescoring(dbox_data=detection_boxes_data_2))

**For checking methods, execute the following lines:**

In [5]:
if __name__ == '__main__':
    # set to print upto 4 precision points
    torch.set_printoptions(precision=4)
    # 
    check_soft_nms()

Linear
tensor([[4.0000, 4.0000, 6.0000, 7.0000, 0.9000],
        [2.0000, 2.0000, 5.0000, 6.0000, 0.8000],
        [3.0000, 1.0000, 5.0000, 5.0000, 0.0478]])
tensor([[1.0000e+02, 1.0000e+02, 5.0000e+02, 4.0000e+02, 8.5000e-01],
        [4.5000e+02, 3.5000e+02, 7.0000e+02, 6.0000e+02, 4.5000e-01],
        [6.0000e+02, 1.0000e+02, 8.0000e+02, 3.0000e+02, 2.0000e-01]])
Gaussian
tensor([[4.0000, 4.0000, 6.0000, 7.0000, 0.9000],
        [2.0000, 2.0000, 5.0000, 6.0000, 0.7192],
        [3.0000, 1.0000, 5.0000, 5.0000, 0.0546]])
tensor([[1.0000e+02, 1.0000e+02, 5.0000e+02, 4.0000e+02, 8.5000e-01],
        [4.5000e+02, 3.5000e+02, 7.0000e+02, 6.0000e+02, 4.4981e-01],
        [6.0000e+02, 1.0000e+02, 8.0000e+02, 3.0000e+02, 2.0000e-01]])


### <font style="color:green">Observations</font>

Let's consider the first detection points. 

Rectangles are listed in the sorted order of their score (max score first).

---

<div>
    <table>
        <tr><td><h3>Detected Bboxes (score)</h3></td> <td><h3>Linear Score</h3></td> <td><h3>Gaussian Score</h3></td> </tr>
        <tr><td><h3>[4, 4, 6, 7] (0.9)</h3></td> <td><h3>0.9</h3></td> <td><h3>0.9</h3></td> </tr>
        <tr><td><h3>[2, 2, 5, 6] (0.8)</h3></td> <td><h3>0.8</h3></td> <td><h3>0.7192</h3></td> </tr>
        <tr><td><h3>[3, 1, 5, 5] (0.1)</h3></td> <td><h3>0.0478</h3></td> <td><h3>0.0546</h3></td> </tr>
    </table>
</div>

- As 0.9 is the maximum score, so it will remain the same.


- The second rectangle is overlapping the first rectangle, but IoU is less than the IoU threshold (`0.3`), so its score does not change by linear Soft-NMS. Gaussian Soft-NMS is not dependent on IoU threshold, but just IoU and sigma value. So the second rectangle score is penalized by Gaussian `Soft-NMS`.


- The third rectangle's IoU with the second rectangle exceeds the IoU threshold (`0.3`), so it is penalized by linear Soft-NMS as well. As it  changes from `0.1` to `0.0478`, we can say IoU should be `1-0.478`.